In [40]:
# Dependencies
import json
import requests
import pandas as pd
import plotly.offline as off
#off.init_notebook_mode(connected=True)
import plotly.plotly as py
off.init_notebook_mode(connected=True)

# Google Developer API Key
from config import gkey

In [41]:
# Census Data Collected from csv's
income = pd.read_csv("Resources/income.csv")
age = pd.read_csv("Resources/age.csv")

In [42]:
# Setup Dataframe to Collect Data from CSVs
stats_df = pd.DataFrame()

stats_df["City-State"] = age["Geography"]
stats_df["Total Population"] = age["Total; Estimate; Total population"]
stats_df["Target Population"] = age["Total; Estimate; AGE - 20 to 24 years"] + age["Total; Estimate; AGE - 25 to 29 years"] + age["Total; Estimate; AGE - 30 to 34 years"] + age["Total; Estimate; AGE - 30 to 34 years"] + age["Total; Estimate; AGE - 35 to 39 years"]
stats_df["Median Income"] = income["Median income (dollars); Estimate; HOUSEHOLD INCOME BY AGE OF HOUSEHOLDER - 25 to 44 years"].map(
    "${:,.2f}".format)
stats_df.head()

,City-State,Total Population,Target Population,Median Income
0,"Abilene, TX Metro Area",168774,36.9,"$53,963.00"
1,"Akron, OH Metro Area",703168,31.4,"$57,276.00"
2,"Albany, GA Metro Area",154830,32.6,"$37,589.00"
3,"Albany, OR Metro Area",119862,31.4,"$53,612.00"
4,"Albany-Schenectady-Troy, NY Metro Area",879291,32.5,"$68,171.00"


In [43]:
# use census MSA lat/long in gAPI
# variables for x (lat) and y (long)
city_df = stats_df
keywords=["brew", "brewpub", "brewery"]
lat = []
long = []
results = []

#Format Everything
city_df["City-URL-String"] = city_df["City-State"].str.replace("Metro Area","")
city_df["City-URL-String"] = city_df["City-URL-String"].str.replace(" ","+")
city_df

,City-State,Total Population,Target Population,Median Income,City-URL-String
0,"Abilene, TX Metro Area",168774,36.9,"$53,963.00","Abilene,+TX+"
1,"Akron, OH Metro Area",703168,31.4,"$57,276.00","Akron,+OH+"
2,"Albany, GA Metro Area",154830,32.6,"$37,589.00","Albany,+GA+"
3,"Albany, OR Metro Area",119862,31.4,"$53,612.00","Albany,+OR+"
4,"Albany-Schenectady-Troy, NY Metro Area",879291,32.5,"$68,171.00","Albany-Schenectady-Troy,+NY+"
5,"Albuquerque, NM Metro Area",904486,34.3,"$50,756.00","Albuquerque,+NM+"
6,"Alexandria, LA Metro Area",154745,32.8,"$44,119.00","Alexandria,+LA+"
7,"Allentown-Bethlehem-Easton, PA-NJ Metro Area",830737,29.9,"$66,996.00","Allentown-Bethlehem-Easton,+PA-NJ+"
8,"Altoona, PA Metro Area",125917,29.1,"$54,733.00","Altoona,+PA+"
9,"Amarillo, TX Metro Area",260495,35.6,"$54,958.00","Amarillo,+TX+"


In [44]:
#Collects the API Results for Each City
for row in range(0,len(city_df)):
    city = city_df["City-URL-String"].loc[row]
    
    #Prints what current city is being called by the API
    print(city)
    
    #Create String for API
    url_string_call =  f"https://maps.googleapis.com/maps/api/geocode/json?address={city}&key={gkey}"
    brewery_data = requests.get(url_string_call)
    
    #Formats Latitude and Longitude Data for Second API Call
    test_var = brewery_data.json()
    target_city = test_var["results"][0]['geometry']['location']
    target_coords = f"{target_city['lat']},{target_city['lng']}"
    params = {
        "location": target_coords,
        "radius": 8000,
        "keyword": keywords,
        "key": gkey
    }
    
    #Second API Call
    base_url = "https://maps.googleapis.com/maps/api/place/radarsearch/json"
    brewery_data = requests.get(base_url, params=params)
    brewery_data = brewery_data.json()

    # Saves Results Data into list to be converted into Dataframes 
    results.append(len(brewery_data["results"]))
    lat.append(target_city['lat'])
    long.append(target_city['lng'])
    #nu_of_relavent_results=len(brewery_data["results"])
    #nu_of_relavent_results

Abilene,+TX+
Akron,+OH+
Albany,+GA+
Albany,+OR+
Albany-Schenectady-Troy,+NY+
Albuquerque,+NM+
Alexandria,+LA+
Allentown-Bethlehem-Easton,+PA-NJ+
Altoona,+PA+
Amarillo,+TX+
Ames,+IA+
Anchorage,+AK+
Ann+Arbor,+MI+
Anniston-Oxford-Jacksonville,+AL+
Appleton,+WI+
Asheville,+NC+
Athens-Clarke+County,+GA+
Atlanta-Sandy+Springs-Roswell,+GA+
Atlantic+City-Hammonton,+NJ+
Auburn-Opelika,+AL+
Augusta-Richmond+County,+GA-SC+
Austin-Round+Rock,+TX+
Bakersfield,+CA+
Baltimore-Columbia-Towson,+MD+
Bangor,+ME+
Barnstable+Town,+MA+
Baton+Rouge,+LA+
Battle+Creek,+MI+
Bay+City,+MI+
Beaumont-Port+Arthur,+TX+
Beckley,+WV+
Bellingham,+WA+
Bend-Redmond,+OR+
Billings,+MT+
Binghamton,+NY+
Birmingham-Hoover,+AL+
Bismarck,+ND+
Blacksburg-Christiansburg-Radford,+VA+
Bloomington,+IL+
Bloomington,+IN+
Bloomsburg-Berwick,+PA+
Boise+City,+ID+
Boston-Cambridge-Newton,+MA-NH+
Boulder,+CO+
Bowling+Green,+KY+
Bremerton-Silverdale,+WA+
Bridgeport-Stamford-Norwalk,+CT+
Brownsville-Harlingen,+TX+
Brunswick,+GA+
Buffalo-Chee

In [46]:
city_df['results']=pd.DataFrame({'results':results})
city_df['lat']=pd.DataFrame({'lat':lat})
city_df['long']=pd.DataFrame({'long':long})
city_df['Ratio']=city_df['Target Population']/(city_df['results'])
city_clean_df = city_df[city_df['results'] != 0]
df = city_clean_df
city_clean_df.sort_values(['results']).head()

,City-State,Total Population,Target Population,Median Income,City-URL-String,results,lat,long,Ratio
146,"Hagerstown-Martinsburg, MD-WV Metro Area",259744,31.5,"$62,872.00","Hagerstown-Martinsburg,+MD-WV+",1,39.606152,-78.019539,31.500
297,"Sacramento--Roseville--Arden-Arcade, CA Metro ...",2242542,34.2,"$63,725.00","Sacramento--Roseville--Arden-Arcade,+CA+",1,39.042681,-120.712002,34.200
271,"Pine Bluff, AR Metro Area",94741,32.4,"$37,098.00","Pine+Bluff,+AR+",3,34.228431,-92.003195,10.800
46,"Bridgeport-Stamford-Norwalk, CT Metro Area",941618,29.7,"$91,003.00","Bridgeport-Stamford-Norwalk,+CT+",4,41.255965,-73.370870,7.425
20,"Augusta-Richmond County, GA-SC Metro Area",584964,33.8,"$47,586.00","Augusta-Richmond+County,+GA-SC+",4,33.362350,-82.040714,8.450


In [84]:
df['text'] = city_clean_df['City-State'] + '<br>Population ' + (city_clean_df['Total Population']/1e6).astype(str)+' million'

limits = [(0,2),(3,10),(11,20),(21,50),(50,3000)]
colors = ["rgb(0,116,217)","rgb(255,65,54)","rgb(133,20,75)","rgb(255,133,27)","lightgrey"]
cities = []
scale = 10

/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [85]:
i=0
lim = limits[i]
df_sub = df
#print(df_sub)
df_sub['scale']=((df_sub['Target Population']/scale)*25)
city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['scale'],
        color = colors[0],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'City'.format(lim[0],lim[1]) )
cities.append(city)

/home/chuck/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy



In [86]:
layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [87]:
i=0
lim = limits[i]
df_sub = df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['results'],
        color = colors[1],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Google Map Results'.format(lim[0],lim[1]) )
cities.append(city)

layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [93]:
i=0
lim = limits[i]
df_sub = df

city = dict(
    type = 'scattergeo',
    locationmode = 'USA-states',
    lon = df_sub['long'],
    lat = df_sub['lat'],
    text = df_sub['text'],
    marker = dict(
        size = df_sub['Ratio']*100,
        color = colors[2],
        line = dict(width=0.5, color='rgb(40,40,40)'),
        sizemode = 'area'
    ),
    name = 'Ratio of Population by Results'.format(lim[0],lim[1]) )


cities.append(city)
#print(df_sub['Ratio']*100)

In [94]:
layout = dict(
        title = '2014 US city populations<br>(Click legend to toggle traces)',
        showlegend = True,
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showland = True,
            landcolor = 'rgb(217, 217, 217)',
            subunitwidth=1,
            countrywidth=1,
            subunitcolor="rgb(255, 255, 255)",
            countrycolor="rgb(255, 255, 255)"
        ),
    )

fig = dict( data=cities, layout=layout )
off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [90]:
# i=0
# lim = limits[i]
# df_sub = df

# city = dict(
#     type = 'scattergeo',
#     locationmode = 'USA-states',
#     lon = df_sub['long'],
#     lat = df_sub['lat'],
#     text = df_sub['text'],
#     marker = dict(
#         size = df_sub['results'],
#         color = colors[i],
#         line = dict(width=0.5, color='rgb(40,40,40)'),
#         sizemode = 'area'
#     ),
#     name = '{0} - {1}'.format(lim[0],lim[1]) )
# cities.append(city)

# layout = dict(
#         title = '2014 US city populations<br>(Click legend to toggle traces)',
#         showlegend = True,
#         geo = dict(
#             scope='usa',
#             projection=dict( type='albers usa' ),
#             showland = True,
#             landcolor = 'rgb(217, 217, 217)',
#             subunitwidth=1,
#             countrywidth=1,
#             subunitcolor="rgb(255, 255, 255)",
#             countrycolor="rgb(255, 255, 255)"
#         ),
#     )

# fig = dict( data=cities, layout=layout )
# off.iplot( fig, validate=False, filename='d3-bubble-map-populations' )

In [91]:
df.sort_values(['Ratio'], ascending=False).head()

,City-State,Total Population,Target Population,Median Income,City-URL-String,results,lat,long,Ratio,text,scale
297,"Sacramento--Roseville--Arden-Arcade, CA Metro ...",2242542,34.2,"$63,725.00","Sacramento--Roseville--Arden-Arcade,+CA+",1,39.042681,-120.712002,34.200,"Sacramento--Roseville--Arden-Arcade, CA Metro ...",85.50
146,"Hagerstown-Martinsburg, MD-WV Metro Area",259744,31.5,"$62,872.00","Hagerstown-Martinsburg,+MD-WV+",1,39.606152,-78.019539,31.500,"Hagerstown-Martinsburg, MD-WV Metro Area<br>Po...",78.75
271,"Pine Bluff, AR Metro Area",94741,32.4,"$37,098.00","Pine+Bluff,+AR+",3,34.228431,-92.003195,10.800,"Pine Bluff, AR Metro Area<br>Population 0.0947...",81.00
20,"Augusta-Richmond County, GA-SC Metro Area",584964,33.8,"$47,586.00","Augusta-Richmond+County,+GA-SC+",4,33.362350,-82.040714,8.450,"Augusta-Richmond County, GA-SC Metro Area<br>P...",84.50
46,"Bridgeport-Stamford-Norwalk, CT Metro Area",941618,29.7,"$91,003.00","Bridgeport-Stamford-Norwalk,+CT+",4,41.255965,-73.370870,7.425,"Bridgeport-Stamford-Norwalk, CT Metro Area<br>...",74.25
